# 1. Imports and Configuration (Cell 1)
This cell imports all necessary libraries (threading, hardware control, vision, math, Flask) and defines the key configuration variables for sensor pins and model paths.

In [1]:
import threading
import time
from datetime import datetime
from math import radians, cos, sin, asin, sqrt
import os
import json

from flask import Flask, Response, jsonify, request, render_template_string

# Hardware & vision
from jetbot import Robot, Camera
import cv2
import numpy as np
import Jetson.GPIO as GPIO
import serial
import torch
import torch.nn.functional as F
import torchvision.models as models

print("Imports loaded.")

# ===========================
# CONFIG – customize here
# ===========================

# Ultrasonic sensor pins (BCM numbering)
ULTRA_TRIG_PIN = 26
ULTRA_ECHO_PIN = 25
ULTRA_MIN_CM = 20.0      # stop/avoid if obstacle closer than this

# GPS serial port
GPS_PORT = "/dev/ttyTHS1"
GPS_BAUDRATE = 9600

# Paths for models
HOME = os.path.expanduser("~")

BASE_DIR = os.getcwd()
MODELS_DIR = "/workspace/jetbot/notebooks/patrol_robot/models"

# Collision avoidance model (AlexNet)
COLLISION_MODEL_PATH = os.path.join(MODELS_DIR, "best_model.pth")
COLLISION_THRESHOLD = 0.5    # prob_blocked > this => consider blocked

# MobileNetSSD object detector
SSD_PROTO = MODELS_DIR + "/MobileNetSSD_deploy.prototxt"
SSD_MODEL = MODELS_DIR + "/MobileNetSSD_deploy.caffemodel"
SSD_CONF_THRESH = 0.5

print("BASE_DIR:", BASE_DIR)
print("MODELS_DIR:", MODELS_DIR)
print("COLLISION_MODEL_PATH:",COLLISION_MODEL_PATH)
print("Exists:", os.path.exists(COLLISION_MODEL_PATH))
print("SSD_PROTO:",SSD_PROTO)
print("SSD_MODEL:",SSD_MODEL)

WARNNIG: Jetson.GPIO library has not been verified with this carrier board,


Imports loaded.
BASE_DIR: /workspace/jetbot/notebooks/patrol_robot
MODELS_DIR: /workspace/jetbot/notebooks/patrol_robot/models
COLLISION_MODEL_PATH: /workspace/jetbot/notebooks/patrol_robot/models/best_model.pth
Exists: True
SSD_PROTO: /workspace/jetbot/notebooks/patrol_robot/models/MobileNetSSD_deploy.prototxt
SSD_MODEL: /workspace/jetbot/notebooks/patrol_robot/models/MobileNetSSD_deploy.caffemodel


# 1.2 Function for Return Home

In [2]:
import math

def bearing_deg(lat1, lon1, lat2, lon2):
    """Compute bearing from (lat1, lon1) to (lat2, lon2) in degrees 0-360."""
    lat1 = math.radians(lat1)
    lat2 = math.radians(lat2)
    diff_lon = math.radians(lon2 - lon1)

    y = math.sin(diff_lon) * math.cos(lat2)
    x = math.cos(lat1)*math.sin(lat2) - math.sin(lat1)*math.cos(lat2)*math.cos(diff_lon)

    brng = math.degrees(math.atan2(y, x))
    return (brng + 360) % 360

def normalize_angle(angle):
    """Return angle in range -180 to +180."""
    while angle > 180:
        angle -= 360
    while angle < -180:
        angle += 360
    return angle


# 2. Global State and Helper Functions (Cell 2)
This block sets up the shared STATE dictionary (protected by state_lock) and defines all utility functions, including sensor readers and movement commands, making them available for the next cells.

In [3]:
state_lock = threading.Lock()

STATE = {
    "mode": "IDLE",            # IDLE, ROUTE1, ROUTE2, RETURN_HOME
    "gps_start": None,         # (lat, lon)
    "gps_current": None,       # (lat, lon)
    "distance_to_start": None, # meters
    "ultrasonic_cm": None,
    "prob_blocked": None,
    "last_frame": None,        # numpy BGR frame
    "alerts": [],              # list of dicts with snapshot bytes
    "next_alert_id": 1,
    "should_exit": False,
    "collision_enabled": True,  # ใช้โมเดลชนอยู่ไหม (เปิด/ปิดได้จาก dashboard)
}

# --- GPS / Math Helpers ---

def haversine_m(lat1, lon1, lat2, lon2):
    """Return distance in meters between two GPS coordinates."""
    R = 6371000.0
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2.0)**2 + cos(lat1)*cos(lat2)*sin(dlon/2.0)**2
    c = 2 * asin(sqrt(a))
    return R * c

def parse_nmea_lat_lon(nmea_lat, lat_dir, nmea_lon, lon_dir):
    """Convert NMEA lat/lon (ddmm.mmmm) to decimal degrees."""
    if not nmea_lat or not nmea_lon:
        return None
    try:
        lat_deg = float(nmea_lat[:2]); lat_min = float(nmea_lat[2:])
        lat = lat_deg + lat_min / 60.0
        if lat_dir == 'S':
            lat = -lat

        lon_deg = float(nmea_lon[:3]); lon_min = float(nmea_lon[3:])
        lon = lon_deg + lon_min / 60.0
        if lon_dir == 'W':
            lon = -lon

        return (lat, lon)
    except Exception:
        return None

# --- Sensor / Movement Helpers ---

def read_ultrasonic_cm():
    """Read JSN-SR04T/HC-SR04 via logic level converter, return distance in cm."""
    GPIO.output(ULTRA_TRIG_PIN, True)
    time.sleep(0.00001)
    GPIO.output(ULTRA_TRIG_PIN, False)

    start_time = time.time()
    timeout = start_time + 0.02
    while GPIO.input(ULTRA_ECHO_PIN) == 0 and time.time() < timeout:
        start_time = time.time()
    if time.time() >= timeout:
        return None

    stop_time = time.time()
    timeout = stop_time + 0.02
    while GPIO.input(ULTRA_ECHO_PIN) == 1 and time.time() < timeout:
        stop_time = time.time()
    if time.time() >= timeout:
        return None

    elapsed = stop_time - start_time
    # Distance = (time * speed_of_sound) / 2.0. Speed of sound is 34300 cm/s
    distance_cm = (elapsed * 34300.0) / 2.0
    return distance_cm

def stop_robot():
    try:
        robot.stop()
    except Exception:
        pass

def drive_forward(speed=0.3, duration=1.0):
    """
    Drive forward but check ultrasonic while moving.
    Will stop early if an obstacle is too close.
    """
    end_time = time.time() + duration

    while time.time() < end_time:
        # Read ultrasonic locally
        dist = read_ultrasonic_cm()
        with state_lock:
            STATE["ultrasonic_cm"] = dist

        # If obstacle too close, stop immediately
        if dist is not None and dist < ULTRA_MIN_CM:
            print(f"[DRIVE_FORWARD] Obstacle at {dist:.1f} cm, stopping early.")
            stop_robot()
            return

        # Move forward in small chunks
        robot.set_motors(speed, speed)
        time.sleep(0.05)  # 50 ms step

    # Done with full duration
    stop_robot()

def turn_left(speed=0.3, duration=0.5):
    robot.set_motors(-speed, speed)
    time.sleep(duration)
    robot.stop()

def step_rectangle(mode):
    """ROUTE1 = small rectangle, ROUTE2 = big rectangle."""
    forward_time = 3.0 if mode == "ROUTE1" else 5.0

    for _ in range(4):
        drive_forward(0.30, forward_time)
        robot.stop()
        time.sleep(0.3)
        turn_left(0.30, 0.60)
        robot.stop()
        time.sleep(0.2)        

def add_alert(label, explanation, gps, snapshot_bgr):
    with state_lock:
        alert_id = STATE["next_alert_id"]
        STATE["next_alert_id"] += 1

    _, jpeg_bytes = cv2.imencode(".jpg", snapshot_bgr)
    jpeg_bytes = jpeg_bytes.tobytes()

    alert = {
        "id": alert_id,
        "time": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "type": label,
        "gps": gps,
        "explanation": explanation,
        "snapshot": jpeg_bytes,
    }
    with state_lock:
        STATE["alerts"].insert(0, alert)
        STATE["alerts"] = STATE["alerts"][:20]

    print("[ALERT]", json.dumps({k: v for k, v in alert.items() if k != "snapshot"}))

# --- AI / Vision Helpers ---

def predict_blocked(frame_bgr):
    """Use AlexNet collision model to get probability of 'blocked'."""
    if collision_model is None:
        return None

    img = cv2.resize(frame_bgr, (224, 224))
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_t = torch.from_numpy(img_rgb).float() / 255.0
    img_t = img_t.permute(2, 0, 1).unsqueeze(0).to(device)  # 1x3x224x224

    with torch.no_grad():
        out = collision_model(img_t)          # [1, 2]
        probs = F.softmax(out, dim=1)
        prob_blocked = float(probs[0, 0].item())  # index 0 = blocked
        return prob_blocked

def detect_fire_regions(frame_bgr):
    """Simple HSV-based fire heuristic (orange/red/yellow areas)."""
    small = cv2.resize(frame_bgr, (160, 120))
    hsv = cv2.cvtColor(small, cv2.COLOR_BGR2HSV)

    # HSV ranges for fire/orange/yellow
    lower1 = np.array([0,   100, 150])
    upper1 = np.array([15,  255, 255])
    lower2 = np.array([15,  100, 150])
    upper2 = np.array([35,  255, 255])

    mask1 = cv2.inRange(hsv, lower1, upper1)
    mask2 = cv2.inRange(hsv, lower2, upper2)
    mask = cv2.bitwise_or(mask1, mask2)

    # Simple morphological operations to clean up the mask
    kernel = np.ones((3, 3), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=1)
    mask = cv2.morphologyEx(mask, cv2.MORPH_DILATE, kernel, iterations=1)

    fire_pixels = cv2.countNonZero(mask)
    total_pixels = mask.shape[0] * mask.shape[1]
    ratio = fire_pixels / float(total_pixels) if total_pixels > 0 else 0.0

    return ratio > 0.10

def run_detection(frame_bgr):
    """
    - MobileNetSSD: detect 'person'
    - HSV: detect 'fire'
    Returns list of dicts: {"label": "person"/"fire", "confidence": float}
    """
    detections = []

    # MobileNetSSD person detection
    if ssd_net is not None:
        blob = cv2.dnn.blobFromImage(
            cv2.resize(frame_bgr, (300, 300)),
            0.007843, (300, 300), 127.5
        )
        ssd_net.setInput(blob)
        out = ssd_net.forward()
        for i in range(out.shape[2]):
            conf = out[0, 0, i, 2]
            if conf < SSD_CONF_THRESH:
                continue
            idx = int(out[0, 0, i, 1])
            MOBILENET_SSD_CLASSES = [
                "background", "aeroplane", "bicycle", "bird", "boat",
                "bottle", "bus", "car", "cat", "chair", "cow", "diningtable", 
                "dog", "horse", "motorbike", "person", "pottedplant", 
                "sheep", "sofa", "train", "tvmonitor",
            ]
            if 0 <= idx < len(MOBILENET_SSD_CLASSES):
                label = MOBILENET_SSD_CLASSES[idx]
                if label == "person":
                    detections.append({"label": "person", "confidence": float(conf)})

    # Fire-like detection
    try:
        if detect_fire_regions(frame_bgr):
            detections.append({"label": "fire", "confidence": 0.9})
    except Exception as e:
        print("Fire detection error:", e)

    return detections

print("Helpers defined.")

Helpers defined.


# 3. Hardware and Model Initialization (Cell 3)
Initializes all physical connections and loads the necessary AI models into the Jetson Nano's GPU memory.

In [4]:
app = Flask(__name__)

# JetBot hardware
robot = Robot()
camera = Camera.instance(width=320, height=240)

# ===========================
# Manual control config
# ===========================
MANUAL_SPEED = 0.35        # tweak to what you used in basic_motion
MANUAL_TURN_SPEED = 0.28   # for left/right/backspin
MANUAL_MOVE_TIME = 0.5     # seconds per tap


# GPIO for ultrasonic
GPIO.setmode(GPIO.BCM)
GPIO.setup(ULTRA_TRIG_PIN, GPIO.OUT)
GPIO.setup(ULTRA_ECHO_PIN, GPIO.IN)
GPIO.output(ULTRA_TRIG_PIN, False)
print("Ultrasonic GPIO initialized.")

# GPS serial
gps_serial = None
try:
    gps_serial = serial.Serial(GPS_PORT, GPS_BAUDRATE, timeout=1.0)
    print(f"[GPS] Opened port {GPS_PORT}")
except Exception as e:
    print("[GPS] Could not open port:", e)

# Torch device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("[Torch] Using device:", device)

# ---- Collision Avoidance Model (AlexNet) ----
collision_model = None
try:
    collision_model = models.alexnet(pretrained=False)
    collision_model.classifier[6] = torch.nn.Linear(4096, 2)

    state = torch.load(COLLISION_MODEL_PATH, map_location=device)
    collision_model.load_state_dict(state)
    collision_model.to(device)
    collision_model.eval()
    print("[Collision] Loaded model from", COLLISION_MODEL_PATH)
except Exception as e:
    print("[Collision] ERROR loading collision model:", e)
    collision_model = None
print("COLLISION_MODEL_PATH:", COLLISION_MODEL_PATH)
print("Model file exists:", os.path.exists(COLLISION_MODEL_PATH))


# ---- MobileNetSSD Object Detector ----
# Re-define classes here for robustness, although defined in run_detection
MOBILENET_SSD_CLASSES = [
    "background", "aeroplane", "bicycle", "bird", "boat",
    "bottle", "bus", "car", "cat", "chair", "cow", "diningtable", 
    "dog", "horse", "motorbike", "person", "pottedplant", 
    "sheep", "sofa", "train", "tvmonitor",
]

ssd_net = None
try:
    if os.path.exists(SSD_PROTO) and os.path.exists(SSD_MODEL):
        ssd_net = cv2.dnn.readNetFromCaffe(SSD_PROTO, SSD_MODEL)
        print("[Detector] Loaded MobileNetSSD.")
    else:
        print("[Detector] MobileNetSSD files not found in models/")
except Exception as e:
    print("[Detector] ERROR loading MobileNetSSD:", e)
    ssd_net = None

print("Initialization done.")

Ultrasonic GPIO initialized.
[GPS] Opened port /dev/ttyTHS1
[Torch] Using device: cuda
[Collision] Loaded model from /workspace/jetbot/notebooks/patrol_robot/models/best_model.pth
COLLISION_MODEL_PATH: /workspace/jetbot/notebooks/patrol_robot/models/best_model.pth
Model file exists: True
[Detector] Loaded MobileNetSSD.
Initialization done.


🔬 Hardware Verification Tests
These three cells must be run successfully to ensure your motors and sensors are functioning correctly before starting the complex application threads.

# 4. Basic Motor Test (Cell 4 - Verification)

In [5]:
# --- Basic Motor Test ---
import time
print("Starting Basic Motor Test (1 second forward, 1 second turn left)...")

# 1. Drive Forward (0.3 speed)
robot.set_motors(0.3, 0.3)
time.sleep(1.0)
robot.stop()
print("Test 1: Drove forward for 1 second.")

# 2. Turn Left (0.3 speed)
robot.set_motors(-0.3, 0.3)
time.sleep(1.0)
robot.stop()
print("Test 2: Turned left for 1 second.")

print("Basic Motor Test Complete. Check if the robot moved correctly.")

Starting Basic Motor Test (1 second forward, 1 second turn left)...
Test 1: Drove forward for 1 second.
Test 2: Turned left for 1 second.
Basic Motor Test Complete. Check if the robot moved correctly.


# 5. Ultrasonic Sensor Test (Cell 5 - Verification)

In [6]:
# --- Ultrasonic Test ---
import time

print("Starting Ultrasonic Test. Reading distance 10 times (2 seconds total).")

for i in range(10):
    # Uses the read_ultrasonic_cm() function defined in Cell 2
    dist = read_ultrasonic_cm()
    if dist is None:
        print(f"{i:02d}: distance = None (timeout / invalid)")
    else:
        print(f"{i:02d}: distance = {dist:.1f} cm")
    time.sleep(0.2)

print("Ultrasonic Test Complete. Readings should change if you move your hand in front of the sensor.")

Starting Ultrasonic Test. Reading distance 10 times (2 seconds total).
00: distance = 70.1 cm
01: distance = 56.6 cm
02: distance = 56.5 cm
03: distance = 56.4 cm
04: distance = 56.9 cm
05: distance = 56.6 cm
06: distance = 56.6 cm
07: distance = 41.7 cm
08: distance = 43.0 cm
09: distance = 56.5 cm
Ultrasonic Test Complete. Readings should change if you move your hand in front of the sensor.


# 6. GPS Serial Read Test (Cell 6 - Verification)

In [7]:
# --- GPS Serial Read Test ---
import time

if gps_serial is None:
    print("GPS serial not open. Check GPS_PORT and wiring.")
else:
    print("Reading raw NMEA lines from GPS. Press Stop / Interrupt (■) to stop/interrupt the cell.\n")
    
    got_fix = False
    
    # Try reading for 30 seconds to get a valid fix
    for i in range(60):  
        # Read the line from the initialized serial port
        line = gps_serial.readline().decode(errors="ignore").strip()
        if not line:
            continue
            
        parts = line.split(",")

        # Print GPRMC and GPGGA sentences for verification
        if line.startswith("$GNRMC") or line.startswith("$GPRMC"):
            status = parts[2]
            print(f"{i:02d}: RMC: {line}")
            if status == "A":  # 'A' means Active/Valid fix
                latlon = parse_nmea_lat_lon(parts[3], parts[4], parts[5], parts[6])
                if latlon is not None:
                    print(f"   --> VALID FIX: Lat = {latlon[0]:.6f}, Lon = {latlon[1]:.6f}")
                    got_fix = True
        elif line.startswith("$GNGGA") or line.startswith("$GPGGA"):
             print(f"{i:02d}: GGA: {line}")

        if got_fix:
             print("\nGPS valid fix achieved. Test successful.")
             break

        time.sleep(0.5)

    if not got_fix:
        print("\nNo valid GPS fix yet. This is common indoors. Check raw NMEA output above.")
        
    print("GPS raw test done.")

Reading raw NMEA lines from GPS. Press Stop / Interrupt (■) to stop/interrupt the cell.



SerialException: device reports readiness to read but returned no data (device disconnected or multiple access on port?)

In [8]:
import serial

...

if gps_serial is None:
    print("GPS serial not open. Check GPS_PORT and wiring.")
else:
    print("Reading raw NMEA lines from GPS. Press Stop / Interrupt (■) to stop/interrupt the cell.\n")
    
    got_fix = False
    
    for i in range(60):
        try:
            line = gps_serial.readline().decode(errors="ignore").strip()
        except serial.SerialException as e:
            print("SerialException:", e)
            print("Trying to reopen GPS port...")
            try:
                gps_serial.close()
            except:
                pass
            try:
                gps_serial = serial.Serial(GPS_PORT, GPS_BAUDRATE, timeout=1.0)
                print("Reopened GPS port OK.")
                continue
            except Exception as e2:
                print("Failed to reopen GPS port:", e2)
                break

        if not line:
            continue

        parts = line.split(",")

        if line.startswith("$GNRMC") or line.startswith("$GPRMC"):
            status = parts[2] if len(parts) > 2 else ""
            print(f"{i:02d}: RMC: {line}")
            if status == "A":
                latlon = parse_nmea_lat_lon(parts[3], parts[4], parts[5], parts[6])
                if latlon is not None:
                    print(f"   --> VALID FIX: Lat = {latlon[0]:.6f}, Lon = {latlon[1]:.6f}")
                    got_fix = True

        elif line.startswith("$GNGGA") or line.startswith("$GPGGA"):
            print(f"{i:02d}: GGA: {line}")

        if got_fix:
            print("\nGPS valid fix achieved. Test successful.")
            break

        time.sleep(0.5)

    if not got_fix:
        print("\nNo valid GPS fix yet. This is common indoors. Check raw NMEA output above.")
        
    print("GPS raw test done.")


Reading raw NMEA lines from GPS. Press Stop / Interrupt (■) to stop/interrupt the cell.

SerialException: device reports readiness to read but returned no data (device disconnected or multiple access on port?)
Trying to reopen GPS port...
Reopened GPS port OK.
04: GGA: $GNGGA,145950.00,1404.16090,N,10036.48917,E,2,12,0.82,13.0,M,-28.0,M,,0000*59
15: RMC: $GNRMC,145951.00,A,1404.16095,N,10036.48906,E,0.352,,041225,,,D*6F
   --> VALID FIX: Lat = 14.069349, Lon = 100.608151

GPS valid fix achieved. Test successful.
GPS raw test done.


🛰️ Main Application Threads and Web Interface
These cells define the continuous background operations and launch the web server.

# 7. Thread Functions (Cell 7)
Defines the three independent loops that run concurrently: capturing the camera, reading GPS data, and executing the main control logic.

In [9]:
def camera_loop():
    """Continuously capture frames from JetBot camera."""
    while True:
        with state_lock:
            if STATE["should_exit"]:
                break
        frame = camera.value 
        frame_bgr = (frame[:, :, ::-1] * 255).astype(np.uint8)
        with state_lock:
            STATE["last_frame"] = frame_bgr
        time.sleep(0.03)

def gps_loop():
    """Read NMEA sentences and update GPS state."""
    global gps_serial
    if gps_serial is None:
        return

    while True:
        # Check exit signal
        with state_lock:
            if STATE["should_exit"]:
                break

        try:
            # Read one NMEA line
            line = gps_serial.readline().decode(errors="ignore").strip()

            # Only accept expected sentence types
            if not (
                line.startswith("$GPRMC") or
                line.startswith("$GPGGA") or
                line.startswith("$GNRMC") or
                line.startswith("$GNGGA")
            ):
                continue

            parts = line.split(",")

            # RMC sentence with valid fix
            if (
                (line.startswith("$GPRMC") or line.startswith("$GNRMC"))
                and len(parts) > 6
                and parts[2] == "A"     # 'A' = Active/Valid fix
            ):
                lat_raw = parts[3]
                lat_dir = parts[4]
                lon_raw = parts[5]
                lon_dir = parts[6]

                latlon = parse_nmea_lat_lon(lat_raw, lat_dir, lon_raw, lon_dir)

                if latlon is not None:
                    with state_lock:

                        # Update last_gps BEFORE updating gps_current
                        if STATE.get("gps_current") is not None:
                            STATE["last_gps"] = STATE["gps_current"]
                        else:
                            # First ever valid point
                            STATE["last_gps"] = latlon

                        # Update current GPS
                        STATE["gps_current"] = latlon

                        # Save starting point once
                        if STATE["gps_start"] is None:
                            STATE["gps_start"] = latlon
                        else:
                            # Calculate distance to start
                            lat0, lon0 = STATE["gps_start"]
                            lat1, lon1 = latlon
                            STATE["distance_to_start"] = haversine_m(lat0, lon0, lat1, lon1)

        except Exception as e:
            print(f"[GPS THREAD ERROR] {e}")
            pass

        time.sleep(0.1)

        
def manual_move(cmd, speed=MANUAL_SPEED, duration=MANUAL_MOVE_TIME):
    """
    Fire-and-forget movement used by dashboard buttons.
    Runs in a thread so it won't block Flask.
    """
    global current_mode

    # if you have modes like "auto", "home", etc. force manual
    current_mode = "manual"

    def _do():
        try:
            if cmd == "forward":
                robot.left_motor.value = speed
                robot.right_motor.value = speed
            elif cmd == "backward":
                robot.left_motor.value = -speed
                robot.right_motor.value = -speed
            elif cmd == "left":
                robot.left_motor.value = -MANUAL_TURN_SPEED
                robot.right_motor.value = MANUAL_TURN_SPEED
            elif cmd == "right":
                robot.left_motor.value = MANUAL_TURN_SPEED
                robot.right_motor.value = -MANUAL_TURN_SPEED
            elif cmd == "backspin_left":
                robot.left_motor.value = -MANUAL_TURN_SPEED
                robot.right_motor.value = MANUAL_TURN_SPEED
            elif cmd == "backspin_right":
                robot.left_motor.value = MANUAL_TURN_SPEED
                robot.right_motor.value = -MANUAL_TURN_SPEED
            elif cmd == "go_forward":
                robot.left_motor.value = MANUAL_SPEED
                robot.right_motor.value = MANUAL_SPEED
            elif cmd == "backspin":
                robot.left_motor.value = MANUAL_TURN_SPEED
                robot.right_motor.value = -MANUAL_TURN_SPEED

            elif cmd == "stop":
                robot.stop()
                return
            else:
                print(f"[manual_move] Unknown cmd: {cmd}")
                return

            time.sleep(duration)
        finally:
            robot.stop()

    threading.Thread(target=_do, daemon=True).start()

def control_loop():
    """Main loop: sensors, AI, avoidance & patrol motion."""
    last_det_time = 0

    while True:
        with state_lock:
            if STATE["should_exit"]:
                break
            mode = STATE["mode"]
            last_frame = STATE["last_frame"]
            collision_enabled = STATE.get("collision_enabled", True)

        # 1) Ultrasonic
        dist_cm = read_ultrasonic_cm()
        with state_lock:
            STATE["ultrasonic_cm"] = dist_cm

        # 2) Collision model
#         prob_blocked = None
#         if last_frame is not None:
#             prob_blocked = predict_blocked(last_frame)
#         with state_lock:
#             STATE["prob_blocked"] = prob_blocked
        # 2) Collision model (เฉพาะตอนเปิดใช้งาน)
        prob_blocked = None
        if last_frame is not None and collision_enabled:
            prob_blocked = predict_blocked(last_frame)
        with state_lock:
            STATE["prob_blocked"] = prob_blocked

        # 3) Human + fire detection (rate limited)
        now = time.time()
        if last_frame is not None and (now - last_det_time) > 0.7:
            last_det_time = now
            detections = run_detection(last_frame)
            for det in detections:
                label = det.get("label", "").lower()
                if label in ["person", "fire"]:
                    with state_lock:
                        gps_now = STATE["gps_current"]
                        current_mode = STATE["mode"]
                    explanation = f"{label.capitalize()} detected while mode {current_mode}"
                    add_alert(label, explanation, gps_now, last_frame)
                    stop_robot()
                    time.sleep(0.3)

#         # 4) Avoidance: ultrasonic + AI
#         blocked = False
#         if dist_cm is not None and dist_cm < ULTRA_MIN_CM:
#             blocked = True
#         if prob_blocked is not None and prob_blocked > COLLISION_THRESHOLD:
#             blocked = True

#         if blocked:
#             stop_robot()
#             time.sleep(0.2)
#             # back a bit
#             robot.set_motors(-0.3, -0.3)
#             time.sleep(0.5)
#             robot.stop()
#             # turn away
#             turn_left(0.3, 0.6)
#             continue
        # 4) Avoidance: ultrasonic + AI
        blocked = False
        if dist_cm is not None and dist_cm < ULTRA_MIN_CM:
            blocked = True
        if prob_blocked is not None and prob_blocked > COLLISION_THRESHOLD:
            blocked = True

        # ถ้า IDLE แล้ว เราไม่อยากให้มันวิ่งหนี/ถอยหลังเอง
        if blocked and mode != "IDLE":
            stop_robot()
            time.sleep(0.2)
            # back a bit
            robot.set_motors(-0.3, -0.3)
            time.sleep(0.5)
            robot.stop()
            # small turn
            if np.random.rand() < 0.5:
                robot.set_motors(0.3, -0.3)
            else:
                robot.set_motors(-0.3, 0.3)
            time.sleep(0.5)
            robot.stop()
            continue


        # 5) Mode-based patrol
#         if mode == "ROUTE1" or mode == "ROUTE2":
#             step_rectangle(mode)
#         elif mode == "RETURN_HOME":
#             drive_forward(0.25, 0.5)
#             stop_robot()
            
#             with state_lock:
#                 dist = STATE["distance_to_start"]
            
#             # Check if we are close enough to home (within 2 meters)
#             if dist is not None and dist < 2.0:
#                 stop_robot()
#                 with state_lock:
#                     STATE["mode"] = "IDLE"
#         else:
#             # IDLE
#             stop_robot()
#             time.sleep(0.1)
        # 5) Mode-based patrol
        if mode == "MANUAL":
            # Sensors, AI, avoidance already handled above
            time.sleep(0.05)
            continue
            
        if mode == "ROUTE1" or mode == "ROUTE2":
            step_rectangle(mode)

        elif mode == "RETURN_HOME":

            with state_lock:
                gps_cur = STATE["gps_current"]
                gps_start = STATE["gps_start"]
                last_gps = STATE.get("last_gps")

            # Need at least 2 GPS points to estimate heading
            if gps_cur is None or gps_start is None or last_gps is None:
                # move slowly forward until we get enough GPS data
                drive_forward(0.2, 0.4)
                continue

            cur_lat, cur_lon = gps_cur
            home_lat, home_lon = gps_start
            prev_lat, prev_lon = last_gps

            # Compute target bearing (to home)
            target_br = bearing_deg(cur_lat, cur_lon, home_lat, home_lon)

            # Compute our forward heading (estimated)
            heading = bearing_deg(prev_lat, prev_lon, cur_lat, cur_lon)

            # Compute difference
            error = normalize_angle(target_br - heading)

            # Steering threshold
            if abs(error) < 10:
                # Drive straight toward home
                drive_forward(0.28, 0.4)
            elif error > 0:
                # Need to turn right
                robot.set_motors(0.25, -0.25)
                time.sleep(0.15)
                robot.stop()
            else:
                # Need to turn left
                robot.set_motors(-0.25, 0.25)
                time.sleep(0.15)
                robot.stop()

            # update last_gps
            with state_lock:
                STATE["last_gps"] = gps_cur

            # Stop when close enough (2m)
            with state_lock:
                dst = STATE["distance_to_start"]
            if dst is not None and dst < 2.0:
                stop_robot()
                with state_lock:
                    STATE["mode"] = "IDLE"
                continue

        elif mode == "FREE_ROAM":
            # เดินอิสระ: ขยับไปข้างหน้าทีละนิด
            # การหลบสิ่งกีดขวางใช้ block 4) ข้างบนเหมือนเดิม
            drive_forward(0.25, 1)

        else:
            # IDLE
            stop_robot()
            time.sleep(0.1)


print("Thread functions defined.")

Thread functions defined.


# 8. Flask Endpoints and Dashboard HTML (Cell 8)
Sets up the web streaming and API endpoints and defines the fully functional HTML dashboard template.

In [10]:
def mjpeg_generator():
    """Yield JPEG frames for MJPEG streaming."""
    while True:
        with state_lock:
            frame = STATE["last_frame"]
            if STATE["should_exit"]:
                break
        if frame is None:
            time.sleep(0.1)
            continue
        ret, jpeg = cv2.imencode(".jpg", frame)
        if not ret:
            continue
        jpg_bytes = jpeg.tobytes()
        yield (b"--frame\r\n"
               b"Content-Type: image/jpeg\r\n\r\n" + jpg_bytes + b"\r\n")
        time.sleep(0.05)

@app.route("/video_feed")
def video_feed():
    return Response(mjpeg_generator(),
                    mimetype="multipart/x-mixed-replace; boundary=frame")

@app.route("/status")
def status():
    with state_lock:
        gps_cur = STATE["gps_current"]
        gps_start = STATE["gps_start"]
        status_data = {
            "mode": STATE["mode"],
            "gps_current": gps_cur,
            "gps_start": gps_start,
            "distance_to_start": STATE["distance_to_start"],
            "ultrasonic_cm": STATE["ultrasonic_cm"],
            "prob_blocked": STATE["prob_blocked"],
            "collision_enabled": STATE.get("collision_enabled", True),

        }
    return jsonify(status_data)

@app.route("/alerts")
def alerts():
    with state_lock:
        alerts_simple = [
            {
                "id": a["id"],
                "time": a["time"],
                "type": a["type"],
                "gps": a["gps"],
                "explanation": a["explanation"],
            }
            for a in STATE["alerts"]
        ]
    return jsonify(alerts_simple)

@app.route("/snapshot/<int:alert_id>.jpg")
def snapshot(alert_id):
    with state_lock:
        for a in STATE["alerts"]:
            if a["id"] == alert_id:
                return Response(a["snapshot"], mimetype="image/jpeg")
    return "Not found", 404

# @app.route("/command", methods=["POST"])
# def command():
#     """
#     Accept JSON: {"cmd": "route1" / "route2" / "return home" / "stop"}
#     """
#     data = request.get_json(force=True)
#     raw_cmd = (data.get("cmd") or "").strip().lower()

#     if "route1" in raw_cmd or raw_cmd == "1" or "route 1" in raw_cmd:
#         cmd = "ROUTE1"
#     elif "route2" in raw_cmd or raw_cmd == "2" or "route 2" in raw_cmd:
#         cmd = "ROUTE2"
#     elif "return" in raw_cmd and "home" in raw_cmd:
#         cmd = "RETURN_HOME"
#     elif "stop" in raw_cmd or "idle" in raw_cmd:
#         cmd = "STOP"
#     else:
#         return jsonify({"ok": False, "error": f"Unknown command: {raw_cmd}"}), 400

#     with state_lock:
#         if cmd == "STOP":
#             STATE["mode"] = "IDLE"
#         else:
#             STATE["mode"] = cmd

#     if cmd == "STOP":
#         stop_robot()

#     print("[CMD]", cmd)
#     return jsonify({"ok": True, "mode": STATE["mode"]})
@app.route("/command", methods=["POST"])
def command():
    """
    Accept JSON: {"cmd": "route1" / "route2" / "return home" / "free_roam" / "toggle_ai" / "stop"}
    """
    data = request.get_json(force=True)
    raw_cmd = (data.get("cmd") or "").strip().lower()

    if "route1" in raw_cmd or raw_cmd == "1" or "route 1" in raw_cmd:
        cmd = "ROUTE1"
    elif "route2" in raw_cmd or raw_cmd == "2" or "route 2" in raw_cmd:
        cmd = "ROUTE2"
    elif "return" in raw_cmd and "home" in raw_cmd:
        cmd = "RETURN_HOME"
    elif "free_roam" in raw_cmd or ("free" in raw_cmd and "roam" in raw_cmd):
        cmd = "FREE_ROAM"
    elif "toggle_ai" in raw_cmd or "toggle model" in raw_cmd or "toggle collision" in raw_cmd:
        cmd = "TOGGLE_AI"
    elif "stop" in raw_cmd or "idle" in raw_cmd:
        cmd = "STOP"
    else:
        return jsonify({"ok": False, "error": f"Unknown command: {raw_cmd}"}), 400

    with state_lock:
        if cmd == "STOP":
            STATE["mode"] = "IDLE"
        elif cmd == "TOGGLE_AI":
            STATE["collision_enabled"] = not STATE.get("collision_enabled", True)
        else:
            STATE["mode"] = cmd
        collision_enabled = STATE.get("collision_enabled", True)

    if cmd == "STOP":
        # หยุดมอเตอร์ทันที
        stop_robot()

    return jsonify({"ok": True, "cmd": cmd, "collision_enabled": collision_enabled})

@app.route("/api/move", methods=["POST"])
def api_move():
    data = request.get_json(force=True)
    cmd = data.get("cmd", "")

    print(f"[api_move] cmd = {cmd}")
    manual_move(cmd)

    return jsonify({"ok": True, "cmd": cmd})

@app.route("/set_home", methods=["POST"])
def set_home():
    """Set current GPS location as new home/start point."""
    with state_lock:
        gps_cur = STATE.get("gps_current")
        if gps_cur is None:
            return jsonify({"ok": False, "error": "No GPS fix"}), 400

        STATE["gps_start"] = gps_cur
        STATE["distance_to_start"] = 0.0

    print(f"[SET_HOME] New start point set to {gps_cur}")
    return jsonify({"ok": True, "gps_start": gps_cur})


DASHBOARD_HTML = r"""
<!DOCTYPE html>
<html>
<head>
    <meta charset="utf-8" />
    <title>JetBot Patrol Dashboard</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            background: #111;
            color: #eee;
            margin: 0;
            padding: 8px;
        }

        .row {
            display: flex;
            flex-wrap: wrap;
            gap: 10px;
            margin-bottom: 10px;
        }

        .col {
            flex: 1 1 320px;
            padding: 4px;
            box-sizing: border-box;
        }

        .panel {
            border: 1px solid #444;
            border-radius: 8px;
            padding: 8px;
            background: #222;
        }

        h2 {
            margin-top: 0;
            font-size: 18px;
        }

        #alerts {
            max-height: 250px;
            overflow-y: auto;
        }

        .alert-item {
            border-bottom: 1px solid #555;
            padding: 6px 0;
        }

        button {
            margin: 2px;
            padding: 6px 10px;
            border-radius: 4px;
            border: none;
            background: #2b6cb0;
            color: white;
            cursor: pointer;
            font-size: 14px;
        }

        button:hover {
            background: #2c5282;
        }

        .status-value {
            font-weight: bold;
        }

        .badge {
            padding: 2px 6px;
            border-radius: 4px;
            font-size: 12px;
        }

        /* Manual control pad */
        #control-pad {
            display: grid;
            grid-template-columns: repeat(3, 60px);
            grid-template-rows: repeat(3, 60px);
            gap: 8px;
            justify-content: center;
            align-items: center;
            margin-top: 10px;
        }

        #control-pad button,
        #spin-row button {
            padding: 10px;
            font-size: 18px;
            border-radius: 8px;
            border: 1px solid #444;
            background: #2b6cb0;
        }

        #spin-row {
            display: flex;
            justify-content: center;
            gap: 10px;
            margin-top: 8px;
        }

        .stop-btn {
            background: #f56565;
            color: #fff;
            font-weight: bold;
        }
        
        #extra-row {
            display: flex;
            justify-content: center;
            gap: 10px;
            margin-top: 10px;
        }

        .wide-btn {
            padding: 10px 18px;
            font-size: 16px;
            border-radius: 8px;
            border: 1px solid #444;
            background: #3182ce;
            color: white;
            cursor: pointer;
        }


        /* Responsive: mobile -> single column */
        @media (max-width: 768px) {
            .row {
                flex-direction: column;
            }

            .col {
                flex: 1 1 100%;
            }
        }
    </style>

</head>
<body>
    <h1>JetBot Patrol Dashboard</h1>

    <div class="row">
        <div class="col" style="flex:2;">
            <div class="panel">
                <h2>Live Camera</h2>
                <img id="video" src="/video_feed" style="width:100%; border-radius:8px;"/>
            </div>
        </div>
        <div class="col">
            <div class="panel">
                <h2>GPS Location</h2>
                <p>Current: <span id="gpsCurrent" class="status-value">-</span></p>
                <p>Start: <span id="gpsStart" class="status-value">-</span></p>
                <p>Distance to Start: <span id="gpsDist" class="status-value">-</span></p>
                
                <button onclick="setHome()">Set Current GPS as Start</button>
            </div>
            <div class="panel" style="margin-top:8px;">
                <h2>Ultrasonic & Collision</h2>
                <p>Distance: <span id="ultra" class="status-value">-</span></p>
                <p>Prob Blocked: <span id="blocked" class="status-value">-</span></p>
            </div>
            <div class="panel" style="margin-top:8px;">
                <h2>Mode & Commands</h2>
                <p>
                    Current Mode:
                    <span id="mode" class="status-value">IDLE</span>
                </p>
                <p>
                    Collision Model:
                    <span id="aiStatus" class="status-value">AI: ON</span>
                </p>
                <button onclick="sendCommand('route1')">Route 1 (Small Rect)</button>
                <button onclick="sendCommand('route2')">Route 2 (Big Rect)</button>
                <button onclick="sendCommand('free_roam')">Free Roam</button>
                <button onclick="sendCommand('return home')">Return Home</button>
                <button onclick="sendCommand('toggle_ai')">Toggle AI Model</button>
                <button onclick="sendCommand('stop')">STOP</button>
            </div>

            <!-- NEW: Manual Control -->
            <div class="panel" style="margin-top:8px;">
                <h2>Manual Control</h2>

                <!-- D-pad -->
                <div id="control-pad">
                    <div></div>
                    <button onclick="sendMove('forward')">▲</button>
                    <div></div>

                    <button onclick="sendMove('left')">◀</button>
                    <button onclick="sendMove('stop')" class="stop-btn">■</button>
                    <button onclick="sendMove('right')">▶</button>

                    <div></div>
                    <button onclick="sendMove('backward')">▼</button>
                    <div></div>
                </div>

                <!-- Spin row -->
                <div id="spin-row">
                    <button onclick="sendMove('backspin_left')">⟲ Spin Left</button>
                    <button onclick="sendMove('backspin_right')">⟳ Spin Right</button>
                </div>

                <!-- NEW: Extra buttons below spin row -->
                <div id="extra-row">
                    <button onclick="sendMove('go_forward')" class="wide-btn">Go Forward</button>
                    <button onclick="sendMove('backspin')" class="wide-btn">Back Spin</button>
                </div>
            </div>


        </div>
    </div>

    <div class="row">
        <div class="col" style="flex:1;">
            <div class="panel">
                <h2>Alerts (Human / Fire)</h2>
                <div id="alerts"></div>
            </div>
        </div>
    </div>

<script>
async function fetchStatus() {
    try {
        const res = await fetch('/status');
        const data = await res.json();

        document.getElementById('mode').innerText = data.mode || 'IDLE';

        if (data.gps_current) {
            document.getElementById('gpsCurrent').innerText =
                data.gps_current[0].toFixed(6) + ', ' + data.gps_current[1].toFixed(6);
        } else {
            document.getElementById('gpsCurrent').innerText = '-';
        }

        if (data.gps_start) {
            document.getElementById('gpsStart').innerText =
                data.gps_start[0].toFixed(6) + ', ' + data.gps_start[1].toFixed(6);
        } else {
            document.getElementById('gpsStart').innerText = '-';
        }

        if (data.distance_to_start != null) {
            document.getElementById('gpsDist').innerText =
                data.distance_to_start.toFixed(1) + ' m';
        } else {
            document.getElementById('gpsDist').innerText = '-';
        }

        if (data.ultrasonic_cm != null) {
            document.getElementById('ultra').innerText =
                data.ultrasonic_cm.toFixed(1) + ' cm';
        } else {
            document.getElementById('ultra').innerText = '-';
        }

        if (data.prob_blocked != null) {
            document.getElementById('blocked').innerText =
                data.prob_blocked.toFixed(2);
        } else {
            document.getElementById('blocked').innerText = '-';
        }

        const aiBadge = document.getElementById('aiStatus');
        if (aiBadge && data.collision_enabled !== undefined) {
            aiBadge.innerText = data.collision_enabled ? 'AI: ON' : 'AI: OFF';
        }
    } catch (e) {
        console.error(e);
    }
}

async function fetchAlerts() {
    try {
        const res = await fetch('/alerts');
        const list = await res.json();
        const container = document.getElementById('alerts');
        container.innerHTML = '';
        list.forEach(a => {
            const div = document.createElement('div');
            div.className = 'alert-item';
            const gpsText = a.gps ? (a.gps[0].toFixed(6) + ', ' + a.gps[1].toFixed(6)) : '-';
            div.innerHTML = `
                <div><b>${a.time}</b> –
                    <span class="badge" style="background:#f56565;">${a.type.toUpperCase()}</span>
                </div>
                <div>GPS: ${gpsText}</div>
                <div>${a.explanation}</div>
                <img src="/snapshot/${a.id}.jpg" style="max-width:100%; margin-top:4px; border-radius:4px;"/>
            `;
            container.appendChild(div);
        });
    } catch (e) {
        console.error(e);
    }
}

async function sendCommand(cmd) {
    try {
        const res = await fetch('/command', {
            method: 'POST',
            headers: { 'Content-Type': 'application/json' },
            body: JSON.stringify({ cmd })
        });
        const data = await res.json().catch(() => ({}));

        if (data.collision_enabled !== undefined) {
            const aiBadge = document.getElementById('aiStatus');
            if (aiBadge) {
                aiBadge.innerText = data.collision_enabled ? 'AI: ON' : 'AI: OFF';
            }
        }

        await fetchStatus();
    } catch (e) {
        console.error(e);
    }
}

// NEW: sendMove() -> /api/move
async function sendMove(cmd) {
    try {
        await fetch('/api/move', {
            method: 'POST',
            headers: { 'Content-Type': 'application/json' },
            body: JSON.stringify({ cmd })
        });
    } catch (e) {
        console.error('move error:', e);
    }
}

async function setHome() {
    try {
        const res = await fetch('/set_home', {
            method: 'POST'
        });

        const data = await res.json().catch(() => ({}));

        if (data.ok) {
            alert("Home point updated!\n" +
                  "New Start GPS: " + data.gps_start[0].toFixed(6) + ", " +
                                     data.gps_start[1].toFixed(6));
            fetchStatus(); // Refresh UI
        } else {
            alert("Failed to set home: " + data.error);
        }

    } catch (e) {
        console.error(e);
        alert("Error setting home.");
    }
}


setInterval(fetchStatus, 500);
setInterval(fetchAlerts, 1500);
fetchStatus();
fetchAlerts();
</script>

</body>
</html>
"""


@app.route("/")
def index():
    return render_template_string(DASHBOARD_HTML)

print("Flask endpoints & dashboard set.")

Flask endpoints & dashboard set.


# 9. Start Threads and Flask App (Cell 9)
Starts the background processing and launches the web interface.

In [ ]:
cam_thread = threading.Thread(target=camera_loop, daemon=True)
ctrl_thread = threading.Thread(target=control_loop, daemon=True)
gps_thread = threading.Thread(target=gps_loop, daemon=True)

cam_thread.start()
ctrl_thread.start()
gps_thread.start()

print("Threads started. Launching Flask app...")

try:
    # Run the web server
    app.run(host="0.0.0.0", port=5000, debug=False, threaded=True)
finally:
    # Cleanup on exit (runs when you interrupt the cell)
    with state_lock:
        STATE["should_exit"] = True
    stop_robot()
    GPIO.cleanup()
    if gps_serial is not None:
        gps_serial.close()
    print("Shutting down cleanly.")

Threads started. Launching Flask app...
 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.208.210:5000/ (Press CTRL+C to quit)
192.168.207.61 - - [01/Dec/2025 13:32:37] "GET /status HTTP/1.1" 200 -
192.168.207.61 - - [01/Dec/2025 13:32:37] "GET /alerts HTTP/1.1" 200 -
192.168.207.61 - - [01/Dec/2025 13:32:37] "GET /status HTTP/1.1" 200 -
192.168.207.61 - - [01/Dec/2025 13:32:37] "GET /alerts HTTP/1.1" 200 -
192.168.207.61 - - [01/Dec/2025 13:32:37] "GET /status HTTP/1.1" 200 -
192.168.207.61 - - [01/Dec/2025 13:32:37] "GET /status HTTP/1.1" 200 -
192.168.207.61 - - [01/Dec/2025 13:32:37] "GET /status HTTP/1.1" 200 -
192.168.207.61 - - [01/Dec/2025 13:32:37] "GET /status HTTP/1.1" 200 -
192.168.207.61 - - [01/Dec/2025 13:32:37] "GET /alerts HTTP/1.1" 200 -
192.168.207.61 - - [01/Dec/2025 13:32:38] "GET /alerts HTTP/1.1" 200 -
192.168.207.61 - - [01/Dec/2025 13:32:38] "GET /status HTTP/1.1" 200 -
192.168.207.61 - - [01/Dec/2025 13:32:38] "GET /status HTTP/1.1" 200 -
192.168.207.61 - - [01/Dec/2025 13:32:38] "GET /alert

[ALERT] {"id": 1, "time": "2025-12-01 13:33:24", "type": "person", "gps": [14.0693645, 100.60812566666667], "explanation": "Person detected while mode IDLE"}


192.168.207.61 - - [01/Dec/2025 13:33:25] "GET /alerts HTTP/1.1" 200 -
192.168.205.161 - - [01/Dec/2025 13:33:25] "GET /status HTTP/1.1" 200 -
192.168.207.61 - - [01/Dec/2025 13:33:25] "GET /status HTTP/1.1" 200 -
192.168.207.61 - - [01/Dec/2025 13:33:25] "GET /snapshot/2.jpg HTTP/1.1" 200 -


[ALERT] {"id": 2, "time": "2025-12-01 13:33:25", "type": "person", "gps": [14.0693665, 100.608122], "explanation": "Person detected while mode IDLE"}


...

192.168.205.161 - - [01/Dec/2025 13:42:51] "GET /status HTTP/1.1" 200 -
192.168.205.161 - - [01/Dec/2025 13:42:52] "GET /status HTTP/1.1" 200 -
192.168.205.161 - - [01/Dec/2025 13:42:52] "GET /status HTTP/1.1" 200 -
192.168.205.161 - - [01/Dec/2025 13:42:52] "GET /alerts HTTP/1.1" 200 -
192.168.205.161 - - [01/Dec/2025 13:42:53] "GET /status HTTP/1.1" 200 -
192.168.205.161 - - [01/Dec/2025 13:42:53] "POST /api/move HTTP/1.1" 200 -


[api_move] cmd = backspin_right


192.168.205.161 - - [01/Dec/2025 13:42:53] "GET /status HTTP/1.1" 200 -
192.168.205.161 - - [01/Dec/2025 13:42:54] "GET /status HTTP/1.1" 200 -
192.168.205.161 - - [01/Dec/2025 13:42:54] "GET /alerts HTTP/1.1" 200 -
192.168.205.161 - - [01/Dec/2025 13:42:54] "GET /status HTTP/1.1" 200 -
192.168.205.161 - - [01/Dec/2025 13:42:55] "GET /status HTTP/1.1" 200 -
192.168.205.161 - - [01/Dec/2025 13:42:55] "GET /status HTTP/1.1" 200 -
192.168.205.161 - - [01/Dec/2025 13:42:55] "GET /alerts HTTP/1.1" 200 -
192.168.205.161 - - [01/Dec/2025 13:42:56] "GET /status HTTP/1.1" 200 -
192.168.205.161 - - [01/Dec/2025 13:42:56] "GET /status HTTP/1.1" 200 -
192.168.205.161 - - [01/Dec/2025 13:42:57] "GET /status HTTP/1.1" 200 -
192.168.205.161 - - [01/Dec/2025 13:42:57] "GET /alerts HTTP/1.1" 200 -
192.168.205.161 - - [01/Dec/2025 13:42:57] "GET /status HTTP/1.1" 200 -
192.168.205.161 - - [01/Dec/2025 13:42:58] "GET /status HTTP/1.1" 200 -


[DRIVE_FORWARD] Obstacle at 8.4 cm, stopping early.


192.168.205.161 - - [01/Dec/2025 13:42:58] "GET /status HTTP/1.1" 200 -
192.168.205.161 - - [01/Dec/2025 13:42:58] "GET /alerts HTTP/1.1" 200 -
192.168.205.161 - - [01/Dec/2025 13:42:59] "POST /command HTTP/1.1" 200 -
192.168.205.161 - - [01/Dec/2025 13:42:59] "GET /status HTTP/1.1" 200 -
192.168.205.161 - - [01/Dec/2025 13:42:59] "GET /status HTTP/1.1" 200 -
192.168.205.161 - - [01/Dec/2025 13:42:59] "GET /status HTTP/1.1" 200 -
192.168.205.161 - - [01/Dec/2025 13:43:00] "GET /alerts HTTP/1.1" 200 -
192.168.205.161 - - [01/Dec/2025 13:43:00] "GET /status HTTP/1.1" 200 -
192.168.205.161 - - [01/Dec/2025 13:43:00] "GET /status HTTP/1.1" 200 -
192.168.205.161 - - [01/Dec/2025 13:43:01] "GET /status HTTP/1.1" 200 -
192.168.205.161 - - [01/Dec/2025 13:43:01] "GET /status HTTP/1.1" 200 -
192.168.205.161 - - [01/Dec/2025 13:43:01] "GET /alerts HTTP/1.1" 200 -
192.168.205.161 - - [01/Dec/2025 13:43:02] "GET /status HTTP/1.1" 200 -
192.168.205.161 - - [01/Dec/2025 13:43:02] "GET /status HTTP/1